# OOP Advanced Patterns

Student: Pollux Gronier


B00822392


Fevrier 2026

## Factory Pattern (Registry Implementation)

In [ ]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Dict, Type, Optional

class PaymentProcessor(ABC):
    @abstractmethod
    def validate(self, details: dict) -> Optional[str]:
        pass
    @abstractmethod
    def process(self, amount: float, details: dict) -> dict:
        pass

class CreditCardProcessor(PaymentProcessor):
    def validate(self, details: dict) -> Optional[str]:
        if len(str(details.get("card_number", ""))) != 16: return "Invalid card number"
        if len(str(details.get("cvv", ""))) != 3: return "Invalid CVV"
        return None
    def process(self, amount: float, details: dict) -> dict:
        if err := self.validate(details): return {"success": False, "error": err}
        fee = round(amount * 0.029, 2)
        return {"success": True, "method": "credit_card", "amount": amount + fee, "fee": fee}

class BankTransferProcessor(PaymentProcessor):
    def validate(self, details: dict) -> Optional[str]:
        if len(str(details.get("iban", ""))) < 15: return "Invalid IBAN"
        return None
    def process(self, amount: float, details: dict) -> dict:
        if err := self.validate(details): return {"success": False, "error": err}
        return {"success": True, "method": "bank_transfer", "amount": amount + 1.50, "fee": 1.50}

class PayPalProcessor(PaymentProcessor):
    def validate(self, details: dict) -> Optional[str]:
        if "@" not in str(details.get("email", "")): return "Invalid Email"
        return None
    def process(self, amount: float, details: dict) -> dict:
        if err := self.validate(details): return {"success": False, "error": err}
        fee = round(amount * 0.034 + 0.30, 2)
        return {"success": True, "method": "paypal", "amount": amount + fee, "fee": fee}

class PaymentFactory:
    def __init__(self) -> None:
        self._registry: Dict[str, Type[PaymentProcessor]] = {}
    def register(self, p_type: str, cls: Type[PaymentProcessor]) -> None:
        self._registry[p_type] = cls
    def get_processor(self, p_type: str) -> PaymentProcessor:
        if cls := self._registry.get(p_type): return cls()
        raise ValueError(f"Unknown type: {p_type}")

# Registration & Usage
factory = PaymentFactory()
factory.register("credit_card", CreditCardProcessor)
factory.register("paypal", PayPalProcessor)
factory.register("bank_transfer", BankTransferProcessor)

methods = [
    ("credit_card", {"card_number": "1111222233334444", "cvv": "999"}),
    ("paypal", {"email": "user@provider.com"}),
    ("bank_transfer", {"iban": "FR7612345678901234567890123"})
]

for p_type, data in methods:
    proc = factory.get_processor(p_type)
    res = proc.process(100.0, data)
    print(f"Method:{p_type.upper()} | Total:{res['amount']}€ (Fee:{res.get('fee')}€)")

## Builder Pattern (Immutable Dataclass)

In [ ]:
from dataclasses import dataclass
from typing import Optional

@dataclass(frozen=True)
class Employee:
    first_name: str
    last_name: str
    email: str
    department: str
    position: str
    salary: float
    has_laptop: bool = False
    has_parking: bool = False
    has_vpn_access: bool = False
    has_admin_rights: bool = False

    def __str__(self):
        return f"Employee: {self.first_name} {self.last_name} ({self.position})"

class EmployeeBuilder:
    def __init__(self):
        self._data = {}
    def with_name(self, first: str, last: str) -> EmployeeBuilder:
        self._data['first_name'] = first
        self._data['last_name'] = last
        return self
    def with_email(self, email: str) -> EmployeeBuilder:
        self._data['email'] = email
        return self
    def with_job(self, dept: str, pos: str, sal: float) -> EmployeeBuilder:
        self._data.update({'department': dept, 'position': pos, 'salary': sal})
        return self
    def with_access(self, vpn: bool = False, admin: bool = False) -> EmployeeBuilder:
        self._data.update({'has_vpn_access': vpn, 'has_admin_rights': admin})
        return self
    def with_equipment(self, laptop: bool = False) -> EmployeeBuilder:
        self._data['has_laptop'] = laptop
        return self
    def build(self) -> Employee:
        if 'email' not in self._data: raise ValueError("Email required")
        return Employee(**self._data)

def developer_preset(builder: EmployeeBuilder) -> EmployeeBuilder:
    return builder.with_access(vpn=True, admin=True).with_equipment(laptop=True)

# Tests
worker = (EmployeeBuilder()
          .with_name("John", "Doe")
          .with_email("john@corp.com")
          .with_job("Eng", "Dev", 75000)
          .with_access(vpn=True)
          .build())

dev = developer_preset(EmployeeBuilder().with_name("Jane", "Smith").with_email("jane@corp.com").with_job("Eng", "Senior", 90000)).build()

print(worker)
print(dev)

## Singleton Pattern (Dependency Injection)

In [ ]:
import json
from abc import ABC, abstractmethod

class ConfigSource(ABC):
    @abstractmethod
    def load(self) -> dict: pass

class JsonSource(ConfigSource):
    def load(self) -> dict:
        # Simulating file load for notebook execution
        return {
            "app": {"name": "PayPlat", "debug": True},
            "database": {"host": "localhost", "port": 5432}
        }

class ConfigManager:
    _instance = None
    def __init__(self, source: ConfigSource):
        self._data = source.load()
    
    @classmethod
    def get_instance(cls, source: ConfigSource = None):
        if cls._instance is None and source:
            cls._instance = cls(source)
        return cls._instance
    
    def get(self, path: str):
        val = self._data
        for key in path.split('.'):
            val = val.get(key)
            if val is None: return None
        return val

# Tests
ConfigManager.get_instance(JsonSource()) # Initialize Singleton
cfg = ConfigManager.get_instance()

print(f"DB Host: {cfg.get('database.host')}")
print(f"Same Instance? {cfg is ConfigManager.get_instance()}")